In [1]:
import ISRAnalysis as an

Welcome to JupyROOT 6.14/04


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import sys

# Unfolding for detector (up to dressed lepton level) 

In [3]:
binDef = ("FineCoarse", "FineCoarse") # pt-mass, mass bin
channel = "electron"
#channel_postfix = "detector_dressedDRp1"
channel_postfix = "detector_dressedDRp1_extended"
year = "2016"
doSys = True
regularisation_mode = 0
doInputStat = True
doRMStat = True
doModelUnc = True
ignoreBinZero = True

# Redirect stdout nd stderr
sys.stderr = open("./log/stderr_"+channel+"_"+year+".txt", "w")
sys.stdout = open("./log/stdout_"+channel+"_"+year+".txt", "w")

detUnfold = an.ISRAnalysis("DetUNFOLD", year, channel, regularisation_mode, doInputStat, doRMStat, ignoreBinZero, "matrix", 
                           "Detector_Dressed_DRp1_Fiducial", binDef, channel_postfix, doModelUnc)

# Closure tests
detUnfold_closure = an.ISRAnalysis("DetUNFOLD_Closure", year, channel, regularisation_mode, doInputStat, doRMStat, ignoreBinZero, "matrix", 
                           "Detector_Dressed_DRp1_Fiducial", binDef, channel_postfix, doModelUnc)

detUnfold_closure_2 = an.ISRAnalysis("DetUNFOLD_Closure_2", year, channel, regularisation_mode, doInputStat, doRMStat, ignoreBinZero, "matrix", 
                           "Detector_Dressed_DRp1_Fiducial", binDef, channel_postfix, doModelUnc)

In [4]:
# Nominal
detUnfold.setInputHist(useMCInput=False, unfoldObj=None, dirName="Detector")
detUnfold_closure.setInputHist(useMCInput=True, unfoldObj=None, dirName="Detector")
detUnfold_closure_2.setInputHist(useMCInput=True, unfoldObj=None, dirName="Detector")

In [5]:
detUnfold.setUnfoldBkgs(dirName="Detector")

In [6]:
detUnfold.subFake() # Subract DY fake (out of fiducial volume at gen level), subUnfoldFake()
detUnfold_closure.subFake()
detUnfold_closure_2.subFake()

In [7]:
if doSys == True:
    
    PDF_sys_list = ["PDFerror{:0>3}".format(str(i)) for i in range(1,101)]
    
    if channel == "muon" :
            
        systematics = {
            "Type_2": ["IdSFUp", "IdSFDown", "IsoSFUp","IsoSFDown", "trgSFUp", "trgSFDown", "PUUp", "PUDown", "bvetoUp", "bvetoDown", 
                       "ScaleABUp", "ScaleABDown", "ScaleAUp", "ScaleADown", "ScaleBUp", "ScaleBDown",
                       "LepMomScaleUp","LepMomScaleDown"],
        }
        systematics["Type_2"] += PDF_sys_list
    
    if channel == "electron" :
        
            # 1. Input distribution 2. Background distribution 3. Response matrix
            # 
            # Type 1: Change input(data) and background histogram and response matrix (ex. Lepton scale)
            # Type 2: Change background histograms and response matrix (ex. Lepton ID SF)
            # Type 3: Change only background histotrams (Background normalization)
            # Type 4: Only response matrix changes (ex. Unfolding model systematic or statistics)
            # Type 5: Only unfolding setting changes (for unfolding mode, etc)
            #
            # Lets make a dictionary for all the systematics, and then select for the specific data year
            
            
            systematics = {
            "Type_2": ["IdSFUp", "IdSFDown", "recoSFUp","recoSFDown", "trgSFUp", "trgSFDown", "PUUp", "PUDown", "bvetoUp", "bvetoDown", 
                       "ScaleABUp", "ScaleABDown", "ScaleAUp", "ScaleADown", "ScaleBUp", "ScaleBDown", 
                       #"L1PrefireUp", "L1PrefireDown",
                       "LepMomScaleUp","LepMomScaleDown","LepMomResUp","LepMomResDown"],
            }
            systematics["Type_2"] += PDF_sys_list
            
    for sysType, sysNames in systematics.items():
        for sysName in sysNames:
            print(sysName)
            detUnfold.setSystematics(sysType, sysName) # Currently, response matrix set here
            
            detUnfold.setInputHist(False, None, "Detector", sysType, sysName)
            detUnfold.setUnfoldBkgs("Detector", sysType, sysName) 
            detUnfold.subFake("Detector_DY_Fake", sysType, sysName)
            
            if sysName == "unfoldIterEM" or sysName == "UnfoldingModel" :
                
                detUnfold_closure.setSystematics(sysType, sysName) # Currently, response matrix set here
                
                detUnfold_closure.setInputHist(True, None, "Detector", sysType, sysName)
                detUnfold_closure.subFake("Detector_DY_Fake", sysType, sysName)
                

In [8]:
#detUnfold.checkMatrixCond()

In [9]:
detUnfold.doUnfold() # nominal unfold
detUnfold_closure.doUnfold()
detUnfold_closure_2.doUnfold()

In [10]:
detUnfold.doAcceptance(isFSR=False, outName="DetEfficienyCorr")
detUnfold.closeOutFiles()
detUnfold.combineOutFiles()

detUnfold_closure.closeOutFiles()
detUnfold_closure.combineOutFiles()

detUnfold_closure_2.closeOutFiles()
detUnfold_closure_2.combineOutFiles()

# Unfolding for QED FSR

In [11]:
binDef = ("CoarseCoarse", "CoarseCoarse")
phase_space = "Fiducial"
#channel = "electron"
#year = "2016"
#doSys = True
regularisation_mode = 0
doInputStat = True
doRMStat = True

fsrUnfold = an.ISRAnalysis("FSRUNFOLD", year, channel, regularisation_mode, doInputStat, doRMStat, False, "matrix_fsr", 
                                     "Dressed_DRp1_Dressed_DR4PI_"+phase_space, binDef, channel_postfix, doModelUnc)

fsrUnfold_closure = an.ISRAnalysis("FSRUNFOLD_Closure", year, channel, regularisation_mode, doInputStat, doRMStat, False, "matrix_fsr", 
                                     "Dressed_DRp1_Dressed_DR4PI_"+phase_space, binDef, channel_postfix, doModelUnc)

In [12]:
fsrUnfold.setInputHist(useMCInput=False, unfoldObj = detUnfold, isFSR=True) # Set nominal input using previous unfolding result
fsrUnfold_closure.setInputHist(useMCInput=True, unfoldObj=None, isFSR=True, dirName="Acceptance", inputBinDef=("FineCoarse", "FineCoarse"))

In [13]:
fsrUnfold.subFake(dirName="Dressed_DRp1_DY_Fake", isFSR=True, inputBinDef=("FineCoarse", "FineCoarse"))
fsrUnfold_closure.subFake(dirName="Dressed_DRp1_DY_Fake", isFSR=True, inputBinDef=("FineCoarse", "FineCoarse"))

In [14]:
#fsrUnfold.drawResponseM("Pt")
#fsrUnfold.drawResponseM("Mass")

#fsrUnfold.checkMatrixCond()

In [15]:
'''
Use output from the previous unfold, and use systematic RM
'''
if doSys :
    #fsr_systematics = {
    #                   "Scale":["ScaleABUp", "ScaleABDown", "ScaleAUp", "ScaleADown", "ScaleBUp", "ScaleBDown"], 
    #                   "AlphaS":["AlphaSUp", "AlphaSDown"],
    #                   #"PDF":["PDFerror{:0>3}".format(str(i)) for i in range(1,101)],
    #                   "Unfolding": ["unfoldIterEM"],
    #                   #"FSR": ["fsrPHOTOS","fsrPYTHIA"],
    #}
    fsr_systematics = {
        "Type_1": ["ScaleABUp", "ScaleABDown", "ScaleAUp", "ScaleADown", "ScaleBUp", "ScaleBDown"],
        "Type_4": ["fsrPHOTOS", "fsrPYTHIA"],    
    }
    fsr_systematics["Type_1"] += PDF_sys_list
    
    for sysType, sysNames in fsr_systematics.items():
        for sysName in sysNames:
            print(sysName)
            fsrUnfold.setSystematics(sysType, sysName, True) # Create TUnfoldDensity objects for systematics
            #
            fsrUnfold.setInputHist(False, detUnfold, "", sysType, sysName, isFSR=True)# 
            fsrUnfold.subFake(dirName="Dressed_DRp1_DY_Fake", sys_type=sysType, sys_name=sysName, isFSR=True, inputBinDef=("FineCoarse", "FineCoarse"))

In [16]:
# Use output from the previous unfold, and use default RM
fsrUnfold.setFromPreviousUnfold(detUnfold) # Set using previous unfolding result for systematics

In [17]:
if doSys:
    for _, sysNames in systematics.items():
        for sysName in sysNames:
            
            skip_systematic = False
            for sysType, _ in fsr_systematics.items():
                if sysName in fsr_systematics[sysType] :
                    skip_systematic = True
                    break
            if skip_systematic : continue
                
            fsrUnfold.subFake(dirName="Dressed_DRp1_DY_Fake", sys_type="Type_5", sys_name=sysName, isFSR=True, inputBinDef=("FineCoarse", "FineCoarse"))

In [18]:
fsrUnfold.doUnfold() # do nominal unfold
fsrUnfold_closure.doUnfold() # do nominal unfold

# Acceptance correction 

In [19]:
fsrUnfold.doAcceptance(isFSR=True)

In [20]:
fsrUnfold.closeOutFiles()
fsrUnfold.combineOutFiles()

In [21]:
fsrUnfold_closure.closeOutFiles()
fsrUnfold_closure.combineOutFiles()